In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 11,9

# Домашнее задание к лекции "Визуализация данных"

## Обязательная часть

Вам необходимо провести базовый [EDA](https://en.wikipedia.org/wiki/Exploratory_data_analysis) выбранного набора данных. 

Требования к анализу:
- построить не менее 4 визуализаций различных видов;
- каждая визуализация должным образом оформлена и читается даже в отрыве от контекста; 
- по каждой визуализации необходимо написать вывод (какую гипотезу на ее основе можно выдвинуть?).

**Откуда брать данные?**

Можете взять свои рабочие данные, либо найти открытые данные (например, на [kaggle.com](https://www.kaggle.com/)) по интересующей вас предметной области (тогда не забудьте приложить ссылку на данные). Если идей нет, можете взять один из перечисленных ниже:
- [данные приложений из Google Play](https://www.kaggle.com/lava18/google-play-store-apps);
- [данные о видео из трендов YouTube](https://www.kaggle.com/datasnaek/youtube-new);
- [данные об уровне счастья в разных странах](https://www.kaggle.com/unsdsn/world-happiness).


### Дополнительная часть (необязательная)

* построить дополнительно не менее 2 визуализаций (итого не менее 6);
* в работе должны присутствовать следующие визуализации: boxplot, heatmap, scatter plot matrix;


In [ ]:
df = pd.read_csv('./hw_2/electric')
df.head()

In [ ]:
df.info()

In [ ]:
# График 1 - продажи электромобилей по годам
sales_by_year = df[['Model Year', 'VIN (1-10)']][(df['Model Year']>=2011) & (df['Model Year']<=2022)].groupby('Model Year').count()
sales_by_year.plot(
    kind='bar',
    xlabel='Model Year',
    ylabel='Vin (1-10)',
    title='Sales by year'
)
plt.legend(loc=('upper left'))
for i, val in enumerate(sales_by_year.values):
    plt.text(i, val, int(val), horizontalalignment='center', verticalalignment='bottom', fontdict={'size':12})

In [ ]:
# Количество продаваемых в США электромобилей растёт с 2011 года.

In [ ]:
# График 2 - карта продаж по штатам
# !Т.к. исходные данные "кривые", подавляющее большинство а/м продано в штате Washington, то для целей тренировки эти данные решил исключить
import folium
m = folium.Map(location=[40, -95], zoom_start=4) # initialize the map and store it in a m object
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_geo = f"{url}/us-states.json" # загрузка занных по штатам
sales_by_state = df[['State', 'Model Year', 'VIN (1-10)']][(df['Model Year']>=2011) & (df['Model Year']<=2022) & (df['State']!='WA')].groupby(['State', 'Model Year']).count().reset_index() # датафрейм по продажам
folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=sales_by_state,
    columns=["State", "VIN (1-10)"],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=.1,
    legend_name="Unemployment Rate (%)",
).add_to(m)
folium.LayerControl().add_to(m)
m

In [ ]:
# Электроавтомобили продаются, в основном, в Калифорнии (CA) и Вашингтоне (DC), т.е. в столичном штате на восточном побережье и самом богатом штате на западном. 
# В остальных штатах продажи электроавтомобилей распределены равномерно.
# ! Помним о том, что для наглядности данные были модифицированы вручную.

In [ ]:
# График 3 - распределение по брендам по годам
import seaborn as sns
import numpy as np
sales_by_brand_by_year = df[['Make', 'Model Year', 'VIN (1-10)']][(df['Model Year']>=2011) & (df['Model Year']<=2022)].groupby(['Make', 'Model Year']).count().reset_index()
sales_by_brand_by_year = sales_by_brand_by_year.pivot_table(
    index='Make', columns='Model Year', values='VIN (1-10)', margins=True, aggfunc='sum').sort_values('All', ascending=False).drop('All').drop(columns='All').head(10)
sns.heatmap(sales_by_brand_by_year, cmap='flare', annot=True, fmt=".0f")

In [ ]:
# График 4 - какие бренды из топ-10 сколько продали электроавтомобилей по годам. Как видно, абсолютный либер в последние 5 лет - тесла
sales_tesla_by_model = df[['Model']][df['Make']=='TESLA'].value_counts()
sales_tesla_by_model.plot(
    kind='pie'
    # xlabel='Model Year',
    # ylabel='Model',
    # title='Sales by year'
)

In [ ]:
# какая структура моделей в продажах Тесла? Основные продажи приходятся на две модели Model 3 и Model Y.